# Uniform sampling within a rotational bound

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
from quat_math import (quatAngularDiff, randomQuatNear, random_quaternion, quaternion_about_axis, quaternion_multiply)

# Sampling Parameters

In [ ]:
# Max angular offset to sample
max_angular_offset = np.pi/180*180
num_samples = 1000000

# Random central angle
q0 = random_quaternion()

# Uniform Angle Sampling

Sampling axis from a Gaussian and angle uniformly. This will not uniformly sample the rotational space.

In [ ]:
# Save all angular offsets
angular_offsets_uniform = []

for _ in trange(num_samples):
    axis = np.random.randn(3)
    axis = np.random.randn(3)
    axis /= np.linalg.norm(axis)
    
    angle = np.random.rand() * max_angular_offset
    
    # Uniformly sampled angle quaternion
    q_delta = quaternion_about_axis(angle, axis)
    
    # Rotation central angle by delta quaternion
    q1 = quaternion_multiply(q_delta, q0)
    # Compute angular offset
    ang_diff = quatAngularDiff(q0, q1)

    # Check to make sure the offset is within the bounds
    if ang_diff <= max_angular_offset:
        angular_offsets_uniform.append(ang_diff)
    else:
        # This should never print
        print('Invalid Offset')

# Compute histogram of offset magnitudes
counts_uniform, bins = np.histogram(angular_offsets_uniform, bins = np.linspace(0, max_angular_offset, 100))

# Normalize distribution
prob_uniform = counts_uniform / num_samples

# Rejection Sampling

Sample the whole rotation space and reject all angles greater than the maximum.
This becomes increasingly inefficient as the max angle gets smaller.

In [ ]:
# Save all angular offsets that are below the max angle
angular_offsets_rejection = []

with tqdm(total=num_samples) as pbar:
    while len(angular_offsets_rejection) < num_samples:
        # Randomly sampled quaternion from whole rotation space
        q1 = random_quaternion()
        
        # Compute angular offset
        ang_diff = quatAngularDiff(q0, q1)
        
        # Save if below angular offset threshold
        if ang_diff <= max_angular_offset:
            angular_offsets_rejection.append(ang_diff)
            pbar.update(1)

# Compute histogram of offset magnitudes
counts_rejection, bins = np.histogram(angular_offsets_rejection, bins = np.linspace(0, max_angular_offset, 100))

# Normalize distribution
prob_rejection = counts_rejection / num_samples

# Inverse CDF Sampling

Sample the within the rotation space we care about using inverse cdf sampling to avoid oversampling small angles

In [ ]:
# Sampled angular offsets
angular_offsets_inverse_cdf = []

for _ in trange(num_samples):
    # Sample only rotations within threshold proportional to their distribution in the full rotational space.
    q1,_ = randomQuatNear(q0, max_angular_offset)

    # Compute angular offset
    ang_diff = quatAngularDiff(q0, q1)
    
    # Check to make sure the offset is within the bounds
    if ang_diff <= max_angular_offset:
        angular_offsets_inverse_cdf.append(ang_diff)
    else:
        # This should never print
        print('Invalid Offset')

# Compute histogram of offset magnitudes
counts_inverse_cdf, _ = np.histogram(angular_offsets_inverse_cdf, bins = np.linspace(0, max_angular_offset, 100))

# Normalize distribution
prob_inverse_cdf = counts_inverse_cdf / num_samples

# Analytical Solution

Compute the analytical PDF of each rotation bin. 

In [ ]:
# Analytical pdf
def angular_pdf(theta):
    return np.pi/2. * np.sin(theta/2.)**2

def angular_cdf(theta):
    return 1./2. * (theta - np.sin(theta)) / (np.pi / 2.)

# Angular bin values for histograms
angular_bins = np.linspace(0, max_angular_offset, 100)

# PDF of initial bin value
pdf_analytical = angular_pdf(angular_bins[:-1])

# PDF of bin value as differnce of edge cdfs
pdf_analytical_bin = np.diff(angular_cdf(angular_bins))

# Normalize 
prob_analytical = pdf_analytical / pdf_analytical.sum()
prob_analytical_bin = pdf_analytical_bin / pdf_analytical_bin.sum()

# Sampling Proportions 

In [ ]:
angular_offset = angular_bins[:-1]
plt.plot(angular_offset, prob_rejection, label='Rejection Sampling')
plt.plot(angular_offset, prob_inverse_cdf, label='Inverse CDF Sampling')
plt.plot(angular_offset, prob_uniform, label='Uniform Angle Sampling')
plt.plot(angular_offset, prob_analytical_bin, label='Analytical Solution')

plt.title('Angle Constrained Sampling Test')
plt.ylabel('Proportion of Samples')
plt.xlabel('Angular Offset')
plt.legend()
plt.show()